In [ ]:
from importlib import reload
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import numpy as np
import pickle
import itertools
import multiprocessing as mp
from datetime import date, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
import randomforestanalysis as RFA
from matplotlib import rc

Define data source and file path to data

In [ ]:
# available data sources are CCCSL, WHOPHSM and CORONANET
datasource = "CCCSL"

In [ ]:
# adapt as necessary

if datasource == "CCCSL":
    datapath = '../data/COVID19_data_cumulative_PAPER_VERSION.csv'
    file_ending = ''

else:
    file_ending = '_'+datasource
    datapath = '../data/other_sources/COVID19_data_cumulative_PAPER_VERSION'+file_ending+'.csv'

Initialise random forest analysis object

In [ ]:

rfa = RFA.RandomForestAnalysis(timeshift=[s for s in range(0,21)],
                               n_splits=10,
                               enddate = date(2020,4,30),
                               minsamples_leaf=[1],
                               max_tree_depth = [d for d in range(1,16)],
                               max_features = [k/100 for k in range(1,101)],
                               n_estimators=500,
                               outcome_name = "R",
                               data_path = datapath
                               )

In [ ]:
rfa._measurenames = list(rfa.data.columns[10:-5],)
rfa.get_predictors()
rfa.get_outcome()

Crossvalidation

In [ ]:
rfa.crossvalidate(n_processes=30)

###### Determine best value for hyperparameter m, depending on time shift s

In [ ]:
rfa.get_performance()

Plot heatmap of hyperparameter dependent model performance

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = plt.gca()
cax = plt.imshow(rfa.performance.loc[(10,1,slice(None),slice(None)),[("R2_test","mean")]].droplevel(['timeshift','min_samples_leaf']).unstack(level=1).values[:10,:],aspect=10,vmin=0.40,vmax=0.475)
cbar = fig.colorbar(cax, orientation='vertical',shrink=.825,label="$<r^2>$")
cbar.ax.set_yticklabels(["$\leq$ 0.4","0.41","0.42","0.43","0.44","0.45","0.46","0.47"])
plt.xticks([19,39,59,79,99],[20,40,60,80,100])
plt.yticks(np.arange(1,11,2),np.arange(2,11,2))
ax.set_ylabel('Maximum tree depth $d$')
ax.set_xlabel('Percentage $m$ of features considered')
plt.savefig('crossvalidation_RF_heatmap'+file_ending+'.pdf',bbox_inches="tight")
plt.show()

Get list of countries in each continent

In [ ]:
countries = pd.read_csv('countries.csv',header=0,sep=';',index_col="Country",usecols=["Country","Europe+Africa","Asia+Oceania","Americas"]).fillna(False).replace(1,True)

countrylists = dict()

countrylists["Americas"] = list(countries.loc[countries["Americas"]==True].index)
countrylists["Asia"]     = list(countries.loc[countries["Asia+Oceania"]==True].index)
countrylists["Europe"]   = list(countries.loc[countries["Europe+Africa"]==True].index)
countrylists["None"]     = []

permutation_importances = dict()

Compute permutation importances for main analysis and continent knockout experiments

In [ ]:
t = rfa._timeshift

n_processes = 21
  
def fun(s,rfa,dropcountries):

    p = rfa.get_optimal_parameters("R2_test",s)
    d = p[1]
    m = p[2]
    
    return rfa.get_permutation_importance(time_shift=s,min_samples_leaf=1,max_depth=d,max_features=m,n_splits=10,n_repeats=200,drop_countries=dropcountries)

for continent in ['None','Europe','Asia','Americas']:

    args = list(map(lambda s: (s,rfa,countrylists[continent]),t))

    pool = mp.Pool(processes=n_processes)

    newres = pool.starmap(fun,args)

    pool.close()
    pool.join()

    permutation_importances[continent] = pd.concat(newres)

    rfa.permutation_importances = permutation_importances

Obtain feature ranking for main analysis and different knockout experiments

In [ ]:
dropped_continent = 'None' # change to 'Europe','Asia' or 'Americas'

RFA.feature_ranking(rfa.permutation_importances[dropped_continent])[['Measure','mean_Delta',"CI"]]